# 1. Introdução
Este notebook apresenta diferentes técnicas de suavização de imagens com foco em explicações **detalhadas** e **implementações manuais passo a passo**, seguidas de versões utilizando bibliotecas como OpenCV ou SciPy.

**Observações:**
- As imagens utilizadas são do `skimage.data` (`camera`) ou devem ser fornecidas (`lena.jpg` para OpenCV).
- O foco está na compreensão da aplicação dos **kernels**, sem usar funções prontas no início.
- Cada técnica terá as seguintes etapas:
  1. Teoria e explicação da convolução
  2. Implementação passo a passo com `for` e kernel manual
  3. Implementação com bibliotecas auxiliares (SciPy/OpenCV)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, img_as_float
import cv2
from pathlib import Path

def show_images(img1, img2, title1, title2):

  plt.figure(figsize=(10,4))
  plt.subplot(1,2,1)
  plt.title(title1)
  plt.imshow(img1, cmap='gray')
  plt.subplot(1,2,2)
  plt.title(title2)
  plt.imshow(img2, cmap='gray')
  plt.show()

In [ ]:
# Verifica se já foram baixadas as imagens do drive, baixando-as e descompactando se necessário
! [ ! -d f"/content/lena.png" ] && gdown -O /content/lena.png "16HLcddcqiAv92JsuJ0dbg9NKSz4D2dvj"

image_path = Path("/content/lena.png")

img_cv = cv2.imread(image_path)
img_cv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
img_cv_gray = cv2.cvtColor(img_cv, cv2.COLOR_RGB2GRAY)


# 2. Exemplos

## 2.1. Filtro de Média (Mean Filter)

O filtro de média aplica uma **média aritmética** sobre os pixels de uma vizinhança quadrada. Ele é um caso especial de **convolução 2D**, onde o kernel possui todos os valores iguais:

$$
K = \frac{1}{9}
\begin{bmatrix}
1 & 1 & 1 \\
1 & 1 & 1 \\
1 & 1 & 1
\end{bmatrix}
$$

A aplicação do kernel consiste em mover essa janela sobre a imagem e calcular a média ponderada de cada região. Isso **suaviza** variações abruptas, mas também pode borrar bordas.

In [ ]:
# Importa imagem de exemplo em tons de cinza (512x512) e converte para float64 no intervalo [0, 1]
img = img_as_float(data.camera())

# Define a função que aplica o filtro de média manualmente
def mean_filter_manual(image, kernel_size=3):
    # Calcula o número de pixels de padding (borda) necessário de cada lado
    pad = kernel_size // 2

    # Adiciona padding à imagem replicando os valores das bordas (modo 'edge')
    padded = np.pad(image, pad, mode='edge')

    # Cria uma nova imagem (output) com os mesmos valores e dimensões da original, inicialmente zerada
    output = np.zeros_like(image)

    # Percorre cada pixel da imagem original (excluindo o padding)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            # Extrai a região da imagem ao redor do pixel (janela do tamanho do kernel)
            region = padded[i:i+kernel_size, j:j+kernel_size]

            # Calcula a média dos valores dessa região e atribui ao pixel correspondente na saída
            output[i, j] = np.mean(region)

    # Retorna a imagem suavizada
    return output

# Aplica o filtro de média manual à imagem
mean_manual = mean_filter_manual(img)

# Exibe lado a lado a imagem original e a suavizada
show_images(img, mean_manual, "Original", "Filtro de Média (Manual)")


In [ ]:
img = img_as_float(data.camera())
mean_cv = cv2.blur(img, (3, 3))

show_images(img, mean_cv, "OpenCV: Original",  "OpenCV: Filtro de Média")

## 2.2. Filtro Gaussiano

O filtro Gaussiano suaviza a imagem aplicando uma **função Gaussiana bidimensional** como kernel. Ele dá mais peso aos pixels próximos ao centro e menos aos distantes, reduzindo o efeito de borramento nas bordas.

A função Gaussiana 2D:

$$
G(x, y) = \frac{1}{2\pi\sigma^2} e^{-\frac{x^2 + y^2}{2\sigma^2}}
$$

Esse kernel é usado para aplicar uma **convolução** na imagem. Ele suaviza ruídos leves e preserva melhor as bordas do que o filtro de média.

In [ ]:
def gaussian_kernel(size, sigma=1):
    ax = np.arange(-size // 2 + 1., size // 2 + 1.)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-(xx**2 + yy**2) / (2. * sigma**2))
    return kernel / np.sum(kernel)

def apply_kernel(image, kernel):
    k = kernel.shape[0]
    pad = k // 2
    padded = np.pad(image, pad, mode='reflect')
    output = np.zeros_like(image)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            region = padded[i:i+k, j:j+k]
            output[i, j] = np.sum(region * kernel)
    return output

gaussian_k = gaussian_kernel(5, sigma=1)
gaussian_manual = apply_kernel(img, gaussian_k)

show_images(gaussian_manual, gaussian_k, "Filtro Gaussiano (manual)", "Kernel Gaussiano")

In [ ]:
gaussian_cv = cv2.GaussianBlur(img_cv, (5, 5), sigmaX=1.0)

show_images(img_cv, gaussian_cv, "Original", "Filtro Gaussiano (OpenCV)")

## 2.3. Filtro da Mediana

O filtro da mediana substitui o valor do pixel central pela **mediana** dos valores da vizinhança.

É especialmente eficaz contra ruídos tipo **sal e pimenta**, pois remove valores extremos mantendo as bordas.

$$
I'(x, y) = \text{mediana} \{ I(x+i, y+j) \mid (i,j) \in \mathcal{N} \}
$$

### Implementação manual (aplicação do kernel com `for` em Python puro)

In [ ]:
def median_filter_manual(image, k):
    pad = k // 2
    padded = np.pad(image, pad, mode='edge')
    output = np.zeros_like(image)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            region = padded[i:i+k, j:j+k]
            output[i, j] = np.median(region)
    return output

median_manual = median_filter_manual(img_cv_gray, 7)

show_images(img_cv_gray, median_manual, "Original", "Filtro da Mediana (manual)")

In [ ]:
median_cv = cv2.medianBlur(img_cv_gray, 7)

show_images(img_cv_gray, median_cv, "Original", "Filtro da Mediana (OpenCV)")

## 2.4. Filtro Bilateral

### Teoria
O filtro bilateral suaviza a imagem levando em conta dois pesos:

- $G_s$: distância espacial entre os pixels
- $G_r$: diferença de intensidade

$$
I'(x) = \frac{1}{W_p} \sum_{x_i \in \mathcal{N}} G_s(||x_i - x||) G_r(|I(x_i) - I(x)|) I(x_i)
$$

É um filtro **não linear** que preserva bem as bordas, mas é caro computacionalmente.

### Implementação manual (aplicação do kernel com `for` em Python puro)

In [ ]:
# Filtro bilateral é muito lento para ser implementado com `for`, então faremos apenas com OpenCV.

In [ ]:
bilateral_cv = cv2.bilateralFilter(img_cv_gray, d=11, sigmaColor=75, sigmaSpace=75)

show_images(img_cv_gray, bilateral_cv, "Original", "Filtro Bilateral (OpenCV)")

## 2.5. Filtro de Kuwahara (manual)

O filtro de Kuwahara divide a janela ao redor de cada pixel em 4 regiões (quadrantes), calcula a média e variância em cada uma e escolhe a **média da região com menor variância**.

Ele preserva bordas bem, pois evita regiões de transição ao escolher a região mais homogênea.

In [ ]:
# Define a função que aplica o filtro de Kuwahara manualmente
def kuwahara_filter(image, window_size=5):
    # Calcula o tamanho do padding necessário em torno da imagem
    pad = window_size // 2

    # Adiciona padding refletido à imagem (espelha os valores nas bordas)
    padded = np.pad(image, pad, mode='reflect')

    # Cria imagem de saída com as mesmas dimensões e tipo da imagem original
    output = np.zeros_like(image)

    # Percorre cada pixel da imagem original
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            # Extrai a região local da imagem centrada no pixel atual
            r = padded[i:i+window_size, j:j+window_size]

            # Divide a região em 4 quadrantes:
            quads = [
                r[:pad+1, :pad+1],   # canto superior esquerdo
                r[:pad+1, pad:],     # canto superior direito
                r[pad:, :pad+1],     # canto inferior esquerdo
                r[pad:, pad:]        # canto inferior direito
            ]

            # Calcula a média de intensidade para cada quadrante
            means = [np.mean(q) for q in quads]

            # Calcula a variância de intensidade para cada quadrante
            variances = [np.var(q) for q in quads]

            # Escolhe a média do quadrante com menor variância
            output[i, j] = means[np.argmin(variances)]

    # Retorna a imagem resultante após aplicação do filtro
    return output

# Aplica o filtro de Kuwahara à imagem com janela
kuwahara_manual = kuwahara_filter(img_cv_gray, 11)

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.title("Original")
plt.imshow(img_cv_gray, cmap='gray')
plt.subplot(1,2,2)
plt.title("Filtro de Kuwahara (manual)")
plt.imshow(kuwahara_manual, cmap='gray')
plt.show()